In [1]:
import os
import sys
sys.path.append(os.path.abspath(".."))
import pandas as pd
import torch
from preprocessing import *
import time
from PC_transformer import PCTransformer

# set_seed(42)

In [2]:
df = pd.read_csv(r'..\output_csv\HCMUT-SuperNodeXP-2017-1.0.swf.csv')
# df = df.head(100)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15886 entries, 0 to 15885
Data columns (total 18 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   job_id                    15886 non-null  float64
 1   submit_time               15886 non-null  float64
 2   wait_time                 15886 non-null  float64
 3   run_time                  15886 non-null  float64
 4   num_allocated_processors  15886 non-null  float64
 5   avg_cpu_time_used         15886 non-null  float64
 6   used_memory               15886 non-null  float64
 7   requested_processors      15886 non-null  float64
 8   requested_time            15886 non-null  float64
 9   requested_memory          15886 non-null  float64
 10  status                    15886 non-null  float64
 11  user_id                   15886 non-null  float64
 12  group_id                  15886 non-null  float64
 13  executable_id             15886 non-null  float64
 14  queue_

In [3]:
df.head()

,job_id,submit_time,wait_time,run_time,num_allocated_processors,avg_cpu_time_used,used_memory,requested_processors,requested_time,requested_memory,status,user_id,group_id,executable_id,queue_id,partition_id,preceding_job_id,think_time
0,4.0,0.0,0.0,13.0,24.0,7.0,14761.0,24.0,21600.0,218453.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,-1.0
1,8.0,40819.0,0.0,61.0,24.0,45.0,14761.0,24.0,21600.0,218453.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,-1.0
2,10.0,45255.0,0.0,60.0,24.0,45.0,14760.0,24.0,21600.0,218453.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,-1.0
3,11.0,45782.0,0.0,1.0,24.0,0.0,0.0,24.0,21600.0,218453.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,-1.0
4,12.0,56510.0,0.0,1.0,24.0,0.0,0.0,24.0,21600.0,218453.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,-1.0


In [4]:
feature_columns = ['requested_processors', 'requested_time', 'avg_cpu_time_used', 'used_memory', 'submit_time', 'wait_time', 'user_id', 'group_id', 'executable_id', 'queue_id']
target_column = 'run_time'

# Hyperparameters
input_dim = len(feature_columns)    # Example input dimension
d_model = 64      # Model dimension
num_heads = 8     # Number of attention heads
d_ff = 256        # Feed-forward dimension
num_layers = 3    # Number of transformer blocks
output_dim = 1    # Output dimension
batch_size = 128  # Batch size
seq_len = 20      # Length of data group
num_epochs = 100   # Number of training epochs
dropout = 0.2    # Dropout rate
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

dataloaders, scaler = prepare_data_seq(df, feature_columns, target_column, seq_len=seq_len, batch_size=batch_size)

In [5]:
rmse_lst, mae_lst, mse_lst, r2_lst, infer_time_lst = [], [], [], [], []

for iter in range(101):
    print(f'\nIteration {iter + 1} / 100')
    # Create model
    model = PCTransformer(input_dim, d_model, num_heads, d_ff, num_layers, output_dim, dropout).to(device)

    # Train the model
    model.train_model(model, dataloaders, num_epochs, device, scaler)

    model.load_state_dict(torch.load('../models_HCMUT/best_pc_transformer_model.pth', map_location=device))
    rmse, mae, mse, r2, infer_time = model.evaluate_model(model, dataloaders['test'], device, scaler, input_dim)

    rmse_lst.append(rmse)
    mae_lst.append(mae)
    mse_lst.append(mse)
    r2_lst.append(r2)
    infer_time_lst.append(infer_time)

    print(f'\nFinal Test Metrics after loading best model:')
    print(f'Test RMSE: {rmse:.4f}, Test MAE: {mae:.4f}, Test MSE: {mse:.4f}, Test R2: {r2:.4f}')
    print(f'Inference Time: {infer_time} seconds per sample')

# Save results to CSV
results_df = pd.DataFrame({
    'RMSE': rmse_lst,
    'MAE': mae_lst,
    'MSE': mse_lst,
    'R2': r2_lst,
    'Inference Time': infer_time_lst
})
results_df.to_csv('../output_HCMUT/pc_transformer_results.csv', index=False)
# Final evaluation on test set
# test_rmse, test_mae, test_mse, test_r2 = model.evaluate_model(model, dataloaders['test'], device, scaler, input_dim)
# print(f'\nFinal Test Metrics:')
# print(f'Test RMSE: {test_rmse:.4f}, Test MAE: {test_mae:.4f}, Test MSE: {test_mse:.4f}, Test R2: {test_r2:.4f}')
# print(f'Training time: {end - start:.2f} seconds')


Iteration 1 / 100
Epoch [1/100]
Train Loss: 0.3294, Val Loss: 0.2910
Val RMSE: 144678.2726, Val MAE: 82680.0731, Val MSE: 20931802549.7689, Val R2: -0.0635
Epoch [2/100]
Train Loss: 0.2783, Val Loss: 0.2755
Val RMSE: 141359.8837, Val MAE: 84561.8485, Val MSE: 19982616728.2692, Val R2: -0.0152
Epoch [3/100]
Train Loss: 0.2702, Val Loss: 0.2752
Val RMSE: 142146.0182, Val MAE: 82616.5615, Val MSE: 20205490504.0981, Val R2: -0.0266
Epoch [4/100]
Train Loss: 0.2678, Val Loss: 0.2830
Val RMSE: 141567.1403, Val MAE: 88264.7121, Val MSE: 20041255199.5232, Val R2: -0.0182
Epoch [5/100]
Train Loss: 0.2658, Val Loss: 0.2770
Val RMSE: 143296.0407, Val MAE: 80337.0110, Val MSE: 20533755284.4947, Val R2: -0.0432
Epoch [6/100]
Train Loss: 0.2633, Val Loss: 0.2729
Val RMSE: 142361.2777, Val MAE: 81335.7606, Val MSE: 20266733388.3923, Val R2: -0.0297
Epoch [7/100]
Train Loss: 0.2576, Val Loss: 0.2704
Val RMSE: 141165.7799, Val MAE: 83006.7213, Val MSE: 19927777425.4504, Val R2: -0.0125
Epoch [8/100]
T